In [108]:
import pandas as pd
import numpy as np


In [109]:
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("../EDA/doge_after_EDA.csv")


In [110]:
from sklearn.model_selection import train_test_split

X = df.drop(['close'], axis=1)
y = df['close']

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()


X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  

X_train_cv, X_test, y_train_cv, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_train, X_cv, y_train, y_cv = train_test_split(X_train_cv, y_train_cv, test_size=0.2, random_state=42)

# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# X_cv = X_cv.to_numpy()

# y_train = y_train.to_numpy()
# y_test_array = y_test.to_numpy()
# y_cv_array = y_cv.to_numpy()

print("X train = ", X_train.shape, "; y train  =", y_train.shape)
print("X cv = ", X_cv.shape, "; y cv  =", y_cv.shape)
print("X test = ", X_test.shape, "; y test  =", y_test.shape)


X train =  (1493, 15) ; y train  = (1493, 1)
X cv =  (374, 15) ; y cv  = (374, 1)
X test =  (467, 15) ; y test  = (467, 1)


In [111]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = models.Sequential()

nsample, nfeature = X_train.shape
time_step = 10

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape = (time_step, nfeature)))
model.add(MaxPooling1D(pool_size = 2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(units = 64, activation = 'relu'))

model.add(Dense(1))  # If you are predicting stock prices (regression)

model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_35 (Conv1D)          (None, 8, 64)             2944      
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 4, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_36 (Conv1D)          (None, 2, 128)            24704     
                                                                 
 max_pooling1d_32 (MaxPoolin  (None, 1, 128)           0         
 g1D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 128)               0         
                                                                 
 dense_31 (Dense)            (None, 64)              

In [112]:
import numpy as np

def create_sequences(X, time_step):
    Xs = []
    for i in range(len(X) - time_step):
        Xs.append(X[i:i + time_step])  # Create sequences of time_step length
    return np.array(Xs)

# Reshape X_train, X_cv, and X_test
time_step = 10

# Convert to numpy and reshape X datasets only
X_train = create_sequences(X_train, time_step)
X_cv = create_sequences(X_cv, time_step)
X_test = create_sequences(X_test, time_step)

# Ensure y_train is aligned with X_train
y_train = y_train[time_step:]  # Skip the first 'time_step' values
y_cv = y_cv[time_step:]        # Similarly for validation set
y_test = y_test[time_step:]    # Similarly for test set

# Print the new shapes
print(f"X_train reshaped: {X_train.shape}")
print(f"X_cv reshaped: {X_cv.shape}")
print(f"X_test reshaped: {X_test.shape}")

X_train reshaped: (1483, 10, 15)
X_cv reshaped: (364, 10, 15)
X_test reshaped: (457, 10, 15)


In [113]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_cv, y_cv))

Epoch 1/50
47/47 [==============================] - 0s 3ms/step - loss: 0.0204 - val_loss: 0.0164
Epoch 2/50
47/47 [==============================] - 0s 1ms/step - loss: 0.0187 - val_loss: 0.0167
Epoch 3/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0174 - val_loss: 0.0155
Epoch 4/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0174 - val_loss: 0.0154
Epoch 5/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0169 - val_loss: 0.0151
Epoch 6/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0162 - val_loss: 0.0160
Epoch 7/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0157 - val_loss: 0.0157
Epoch 8/50
47/47 [==============================] - 0s 1ms/step - loss: 0.0155 - val_loss: 0.0156
Epoch 9/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0159 - val_loss: 0.0161
Epoch 10/50
47/47 [==============================] - 0s 2ms/step - loss: 0.0148 - val_loss: 0.0163
Epoch 11/50
47/47 [

In [114]:
from keras.losses import MSE
from sklearn.metrics import mean_squared_error


y_hats = model.predict(X_test)

mse = mean_squared_error(y_test, y_hats)
mse


15/15 [==============================] - 0s 483us/step


0.021448106241556148

In [115]:
y_hats = y_hats.reshape(-1, 1)
y_hats_absolute = scaler_y.inverse_transform(y_hats)
y_hats_absolute.flatten()
y_hats_absolute

array([[ 9.79400054e-02],
       [ 1.07138656e-01],
       [ 1.60207525e-01],
       [ 5.85453361e-02],
       [ 1.15495294e-01],
       [ 8.61639306e-02],
       [ 9.56736207e-02],
       [ 6.34158179e-02],
       [ 5.83307296e-02],
       [ 1.53598353e-01],
       [ 2.34358199e-02],
       [ 9.49524418e-02],
       [ 9.19610187e-02],
       [ 7.39786327e-02],
       [ 6.03672154e-02],
       [ 2.87102908e-01],
       [ 2.42382035e-01],
       [ 2.39747509e-01],
       [ 2.26474598e-01],
       [ 5.68913445e-02],
       [ 6.22071661e-02],
       [ 2.94009540e-02],
       [ 4.11428921e-02],
       [ 2.86030695e-02],
       [ 6.10369956e-03],
       [ 8.82291943e-02],
       [ 6.46720380e-02],
       [ 8.33687931e-02],
       [ 4.23148572e-02],
       [ 1.45868305e-02],
       [ 3.66630428e-03],
       [ 8.86452496e-02],
       [ 2.93581169e-02],
       [ 9.41645503e-02],
       [ 1.85044542e-01],
       [ 3.21602523e-02],
       [ 2.85698026e-01],
       [ 1.03466272e-01],
       [ 2.7

In [135]:
new_df = pd.read_csv("hi.csv")

new_df.head()
X = new_df.drop(['close'], axis = 1)
new_scalar_X = MinMaxScaler()
X_scaled = new_scalar_X.fit_transform(X)
X_scaled_reshaped = X_scaled.reshape(1, 10, 15)

print(X_scaled.shape)
prediction = model.predict(X_scaled_reshaped)


prediction = prediction.reshape(-1, 1)
pred = scaler_y.inverse_transform(prediction)
pred


(10, 15)
1/1 [==============================] - 0s 11ms/step


array([[0.21902044]], dtype=float32)